In [ ]:
# ----------------------------------------------------------
# PROJECT: Data Pipeline for an E-Commerce Company
# CASE STUDY: Integrating Web-Scraped and API Data into SQL
# ==========================================

# The following notebook demonstrates a full data pipeline:
# 1. Web scraping city-level data from Wikipedia
# 2. Extracting geographical coordinates
# 3. Storing and managing data in a MySQL database
# 4. Enriching the dataset with population, weather, and flight information
# ----------------------------------------------------------

In [ ]:
# ==========================================
# STEP 1 — Web Scraping
# ==========================================

# Goal:
# Extract structured city data (name, country, coordinates) from Wikipedia
# and prepare it for insertion into the SQL database.
# ----------------------------------------------------------

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
!pip install lat-lon-parser
from lat_lon_parser import parse
from datetime import datetime

today = datetime.today().strftime("%d.%m.%Y")
print(today)

import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse    


cities = ["Berlin", "Hamburg", "Munich"]
city_data = []
countries = []
latitudes = []
longitudes = []

for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    headers = {'User-Agent': 'Chrome/134.0.0.0'}

    response = requests.get(url, headers=headers)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    country = city_soup.find(class_="infobox-data").get_text()
    city_latitude = parse(city_soup.find(class_="latitude").get_text())
    city_longitude = parse(city_soup.find(class_="longitude").get_text())
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    city_population_clean = city_population.replace(",", "")
    today = datetime.today().strftime("%d.%m.%Y")
    city_data.append({"City": city,
                     "Country": country,
                     "Latitude": city_latitude,
                     "Longitude": city_longitude,
                     "Population": int(city_population_clean),
                     "Population_Timestamp": today})
                 

cities_df = pd.DataFrame(city_data)

cities_main_df = cities_df[["City","Country","Latitude","Longitude" ]]



In [ ]:
# ==========================================
# STEP 2 — SQL Setup
# ==========================================

# Create and configure the MySQL schema that will store
# scraped and API-enriched data.
# ----------------------------------------------------------

# Preparing SQL
# The following SQL commands (executed via Python) create the database
# and the main tables used throughout the project.


# Note:
# The complete SQL schema (with all CREATE TABLE statements and foreign keys)
# is also included in this repository as a separate file: `wiki_cities_full.sql`.
# Refer to that file for the full SQL implementation.

In [ ]:
-- SQL:
-- 1. Create a new database
CREATE DATABASE wiki_cities;
USE wiki_cities;

-- 2. Create a master table for city information
CREATE TABLE city_data (
    city_id INT AUTO_INCREMENT,
    city VARCHAR(50) NOT NULL,
    country VARCHAR(50),
    latitude DECIMAL(8,5),
    longitude DECIMAL(8,5),
    PRIMARY KEY (city_id) 
); 


In [ ]:
# ==========================================
# STEP 3 — Sending Data to SQL
# ==========================================

# The scraped city data is now uploaded from the Jupyter Notebook
# into the MySQL database using SQLAlchemy and pandas.

# Purpose:
# - To insert clean, structured city information into the `city_data` table
# - To ensure that each city receives a unique city_id for later joins
# ----------------------------------------------------------

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv() 
import my_sql_pass

schema = "wiki_cities"
host = "127.0.0.1"
user = "root"
password = my_sql_pass.my_password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

cities_main_df.to_sql('city_data',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [ ]:
# ==========================================
# STEP 4 — Pulling Data Back with City ID
# ==========================================

# After uploading the city data to SQL, retrieve it back into Python
# to confirm successful insertion and to access the generated `city_id` values.
# These IDs are essential for linking population, weather, and flight data
# to their respective cities later in the pipeline.
# ----------------------------------------------------------

In [ ]:
city_data_sql = pd.read_sql("city_data", con=connection_string)
cities_df = cities_df.merge(city_data_sql[["city", "city_id"]],left_on="City", right_on="city", how="left")


In [ ]:
cities_df

In [ ]:
population_df = cities_df[['City', 'Population','Population_Timestamp','city_id']]

In [ ]:
population_df

In [ ]:
# ==========================================
# STEP 5 — Population Data
# ==========================================

# This section creates the SQL table for storing population data
# scraped from Wikipedia and links it to the corresponding city_id.
#
# The population values and timestamps are added later via Python
# once the data is cleaned and formatted.
# ----------------------------------------------------------


In [ ]:
-- SQL:
CREATE TABLE population (
population_row_id INT auto_increment, 
city_id INT, 
City VARCHAR(50),
Population VARCHAR(50), 
Population_Timestamp VARCHAR(50),
PRIMARY KEY (population_row_id), 
FOREIGN KEY (city_id) REFERENCES city_data(city_id)
);

In [ ]:
population_df.to_sql('population',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [ ]:
# ==========================================
# STEP 6 — Weather Data
# ==========================================

# Weather data is collected from the OpenWeather API and stored in SQL.
# Each record includes forecast time, temperature, humidity, and other metrics.
# The `city_id` column ensures correct linkage to the city_data table.
# ----------------------------------------------------------

In [ ]:

import secret_api_key



city_data_sql = pd.read_sql("city_data", con=connection_string)
cities = city_data_sql["city"].tolist()
API_key = secret_api_key.my_api_key
weather_items = []  

for city in cities:  
    weather_data = requests.get(
        f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric"
    )
    weather_data_json = weather_data.json()

    for item in weather_data_json["list"]:
        weather_item = {
            "city": str(city),
            "forecast_time": datetime.strptime(item.get("dt_txt"), "%Y-%m-%d %H:%M:%S"),
            "temperature": float(item["main"].get("temp", None)),
            "feels_like": float(item["main"].get("feels_like", None)),
            "forecast": str(item["weather"][0].get("main", None)),
            "humidity": float(item["main"].get("humidity", None)),
            "clouds": float(item["clouds"].get("all", None)),
            "wind_speed": float(item["wind"].get("speed", None)),
            "rain_in_last_3h": float(item.get("rain", {}).get("3h", 0)),
        }
        weather_items.append(weather_item)


all_cities_df = pd.DataFrame(weather_items)
all_cities_df = all_cities_df.merge(city_data_sql[['city', 'city_id']], on='city', how='left')



In [ ]:
all_flights_df = all_flights_df[["dest_airport_icao","number","movement.airport.icao","movement.airport.name","movement.scheduledTime.utc"]]

In [ ]:
all_flights_df.loc[:, "movement.scheduledTime.utc_dt"] = pd.to_datetime(all_flights_df["movement.scheduledTime.utc"], errors="coerce")
all_flights_df = all_flights_df.rename(columns={
    "movement.airport.icao": "movement_airport_icao",
    "movement.airport.name": "movement_airport_name",
    "movement.scheduledTime.utc_dt": "arrival_time"
})


In [ ]:
all_flights_df = all_flights_df[["dest_airport_icao","number","movement_airport_icao","movement_airport_name","arrival_time"]]

In [ ]:
all_flights_df.to_sql('flights_data',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [ ]:
-- SQL: 
CREATE TABLE weather_cities (
weather_row_id INT auto_increment, 
city_id INT, 
city VARCHAR(50),
forecast_time datetime,
temperature decimal (5,2),
feels_like decimal (5,2),
forecast VARCHAR(50),
humidity decimal (5,2),
clouds decimal (5,2),
wind_speed decimal (5,2),
rain_in_last_3h decimal (5,2),
PRIMARY KEY (weather_row_id), 
FOREIGN KEY (city_id) REFERENCES city_data(city_id)

);

In [ ]:
all_cities_df.to_sql('weather_cities',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [ ]:
# ==========================================
# STEP 7 — Airport Data
# ==========================================

# Airport data is retrieved from the Aerobox API and connected to the city table.
# Each airport record includes ICAO and IATA codes, city name, and airport name.
# ----------------------------------------------------------

In [ ]:
import flights_api_key
API_key_2 = flights_api_key.flights_api_k

all_airports = []
latitudes = city_data_sql["latitude"]
longitudes = city_data_sql["longitude"]

for lat, lon in zip(latitudes, longitudes):
    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat}/{lon}/km/50/16"
    querystring = {"withFlightInfoOnly":"true"}
    headers = {
      "X-RapidAPI-Key": API_key_2,
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
      data = response.json()
      airports = pd.json_normalize(data.get('items', []))
      all_airports.append(airports)

all_airports_df = pd.concat(all_airports, ignore_index=True)


all_airports_df

In [ ]:
all_airports_df = all_airports_df.merge(
    city_data_sql[["city", "city_id"]],
    left_on="municipalityName",
    right_on="city",
    how="left")

In [ ]:
all_airports_df = all_airports_df[["icao", "iata", "city", "city_id", "name"]]

In [ ]:
-- SQL:
CREATE TABLE airports_list (
    airport_id INT AUTO_INCREMENT,
    city_id INT,
    city VARCHAR(50),
    icao VARCHAR(50) UNIQUE, 
    iata VARCHAR(50),
    name VARCHAR(100),
    PRIMARY KEY (airport_id),
    FOREIGN KEY (city_id) REFERENCES city_data(city_id)
);

In [ ]:
all_airports_df.to_sql('airports_list',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [ ]:
# ==========================================
# STEP 8 — Flight Movement Data
# ==========================================

# Flight movement data is also retrieved from the Aerobox API.
# Each record links to the destination airport via ICAO code.
# The table stores flight number, movement airport, and arrival time.
# ----------------------------------------------------------

In [ ]:
all_flights = []
icaos = all_airports_df["icao"]

for icao in icaos:
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/2025-10-17T00:00/2025-10-17T11:59"
    querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"false"}
    headers = {
        "X-RapidAPI-Key": API_key_2,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        flights = pd.json_normalize(data.get('arrivals', []))
        flights["dest_airport_icao"] = icao
        all_flights.append(flights)
    

all_flights_df = pd.concat(all_flights, ignore_index=True)

In [ ]:
all_flights_df = all_flights_df[["dest_airport_icao","number","movement.airport.icao","movement.airport.name","movement.scheduledTime.utc"]]
all_flights_df.loc[:, "movement.scheduledTime.utc_dt"] = pd.to_datetime(all_flights_df["movement.scheduledTime.utc"], errors="coerce")
all_flights_df = all_flights_df.rename(columns={
    "movement.airport.icao": "movement_airport_icao",
    "movement.airport.name": "movement_airport_name",
    "movement.scheduledTime.utc_dt": "arrival_time"
})
all_flights_df = all_flights_df[["dest_airport_icao","number","movement_airport_icao","movement_airport_name","arrival_time"]]

In [ ]:
-- SQL:
CREATE TABLE flights_data (
    flight_id INT AUTO_INCREMENT,
    dest_airport_icao VARCHAR(50),
    `number` VARCHAR(50),
    movement_airport_icao VARCHAR(50),
    movement_airport_name VARCHAR(100),
    arrival_time DATETIME,
    PRIMARY KEY (flight_id),
    FOREIGN KEY (dest_airport_icao) REFERENCES airports_list(icao)
);

In [ ]:
all_flights_df.to_sql('flights_data',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [ ]:
# ==========================================
# STEP 9 — Summary and Next Steps
# ==========================================

# This notebook demonstrates a complete ETL workflow:
# - Web scraping city and population data from Wikipedia
# - Collecting weather, airport, and flight data from external APIs
# - Building and populating a relational MySQL database